# Машинное обучение, ФКН ВШЭ

# Семинар 6

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Работа с текстовыми данными

Разреженные матрицы имеют место в машинном обучении, в частности, в задачах обработки текстов. 

Как правило, модели машинного обучения действуют в предположении, что матрица "объект-признак" является вещественнозначной, поэтому при работе с текстами сперва для каждого из них необходимо составить его признаковое описание. Для этого широко используются техники векторизации, tf-idf и пр. Рассмотрим их на примере [датасета](https://www.dropbox.com/s/f9xsff8xluriy95/banki_responses.json.bz2?dl=0) отзывов о банках.

Сперва загрузим данные:

In [3]:
from tqdm import tqdm
from sklearn.datasets import fetch_20newsgroups

In [4]:
data = fetch_20newsgroups(subset='all', categories=['comp.graphics', 'sci.med'])

Данные содержат тексты новостей, которые надо классифицировать на разделы.

In [5]:
data['target_names']

['comp.graphics', 'sci.med']

In [6]:
texts = data['data']
target = data['target']

Например:

In [7]:
texts[0]

'From: dyer@spdcc.com (Steve Dyer)\nSubject: Re: Analgesics with Diuretics\nOrganization: S.P. Dyer Computer Consulting, Cambridge MA\n\nIn article <ofk=lve00WB2AvUktO@andrew.cmu.edu> Lawrence Curcio <lc2b+@andrew.cmu.edu> writes:\n>I sometimes see OTC preparations for muscle aches/back aches that\n>combine aspirin with a diuretic.\n\nYou certainly do not see OTC preparations advertised as such.\nThe only such ridiculous concoctions are nostrums for premenstrual\nsyndrome, ostensibly to treat headache and "bloating" simultaneously.\nThey\'re worthless.\n\n>The idea seems to be to reduce\n>inflammation by getting rid of fluid. Does this actually work? \n\nThat\'s not the idea, and no, they don\'t work.\n\n-- \nSteve Dyer\ndyer@ursa-major.spdcc.com aka {ima,harvard,rayssd,linus,m2c}!spdcc!dyer\n'

In [8]:
data['target_names'][target[0]]

'sci.med'

### Векторизация

Самый очевидный способ формирования признакового описания текстов — векторизация. Пусть у нас имеется коллекция текстов $D = \{d_i\}_{i=1}^l$ и словарь всех слов, встречающихся в выборке $V = \{v_j\}_{j=1}^d.$ В этом случае некоторый текст $d_i$ описывается вектором $(x_{ij})_{j=1}^d,$ где
$$x_{ij} = \sum_{v \in d_i} [v = v_j].$$

Таким образом, текст $d_i$ описывается вектором количества вхождений каждого слова из словаря в данный текст.

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

In [10]:
vectorizer = CountVectorizer(encoding='utf8', min_df=5)
_ = vectorizer.fit(texts)

Результатом является разреженная матрица.

In [11]:
vectorizer.transform(texts[:1])

<1x7889 sparse matrix of type '<class 'numpy.int64'>'
	with 75 stored elements in Compressed Sparse Row format>

In [12]:
print(vectorizer.transform(texts[:1]).indptr)
print(vectorizer.transform(texts[:1]).indices)
print(vectorizer.transform(texts[:1]).data)

[ 0 75]
[ 563  613  662  743  753  759  869  894  902  918 1025 1076 1334 1367
 1477 1643 1686 1689 1769 1842 2024 2367 2378 2388 2472 2519 3005 3036
 3111 3214 3401 3416 3595 3627 3677 3750 4138 4145 4231 4332 4334 4368
 4752 4885 4912 5016 5057 5106 5128 5817 5825 5892 6109 6112 6337 6344
 6502 6593 6627 6746 6818 6841 6951 7064 7066 7087 7096 7155 7234 7435
 7759 7781 7798 7809 7856]
[1 1 1 1 2 2 1 1 1 1 1 1 1 1 1 2 2 1 1 1 1 1 1 1 6 2 1 2 1 1 1 1 2 1 1 1 1
 1 1 1 1 1 1 1 2 1 1 1 2 1 2 1 1 1 2 1 1 1 3 2 1 2 1 2 3 2 1 3 1 1 2 2 1 1
 1]


### TF-IDF

Ещё один способ работы с текстовыми данными — [TF-IDF](https://en.wikipedia.org/wiki/Tf–idf) (**T**erm **F**requency–**I**nverse **D**ocument **F**requency). Рассмотрим коллекцию текстов $D$.  Для каждого уникального слова $t$ из документа $d \in D$ вычислим следующие величины:

1. Term Frequency – количество вхождений слова в отношении к общему числу слов в тексте:
$$\text{tf}(t, d) = \frac{n_{td}}{\sum_{t \in d} n_{td}},$$
где $n_{td}$ — количество вхождений слова $t$ в текст $d$.
1. Inverse Document Frequency
$$\text{idf}(t, D) = \log \frac{\left| D \right|}{\left| \{d\in D: t \in d\} \right|},$$
где $\left| \{d\in D: t \in d\} \right|$ – количество текстов в коллекции, содержащих слово $t$.

Тогда для каждой пары (слово, текст) $(t, d)$ вычислим величину:
$$\text{tf-idf}(t,d, D) = \text{tf}(t, d)\cdot \text{idf}(t, D).$$

Отметим, что значение $\text{tf}(t, d)$ корректируется для часто встречающихся общеупотребимых слов при помощи значения $\text{idf}(t, D).$

Признаковым описанием одного объекта $d \in D$ будет вектор $\bigg(\text{tf-idf}(t,d, D)\bigg)_{t\in V}$, где $V$ – словарь всех слов, встречающихся в коллекции $D$.

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [14]:
vectorizer = TfidfVectorizer(encoding='utf8', min_df=5)
_ = vectorizer.fit(texts)

На выходе получаем разреженную матрицу.

In [15]:
vectorizer.transform(texts[:1])

<1x7889 sparse matrix of type '<class 'numpy.float64'>'
	with 75 stored elements in Compressed Sparse Row format>

In [16]:
print(vectorizer.transform(texts[:1]).indptr)
print(vectorizer.transform(texts[:1]).indices)
print(vectorizer.transform(texts[:1]).data)

[ 0 75]
[7856 7809 7798 7781 7759 7435 7234 7155 7096 7087 7066 7064 6951 6841
 6818 6746 6627 6593 6502 6344 6337 6112 6109 5892 5825 5817 5128 5106
 5057 5016 4912 4885 4752 4368 4334 4332 4231 4145 4138 3750 3677 3627
 3595 3416 3401 3214 3111 3036 3005 2519 2472 2388 2378 2367 2024 1842
 1769 1689 1686 1643 1477 1367 1334 1076 1025  918  902  894  869  759
  753  743  662  613  563]
[0.03193224 0.03493043 0.1191548  0.11616465 0.06386448 0.10254433
 0.08646835 0.0635189  0.02926056 0.08659203 0.06258286 0.05345822
 0.08196547 0.10988396 0.01897323 0.14127187 0.28780747 0.0817001
 0.11277083 0.06495833 0.11172894 0.11569557 0.10908706 0.09333267
 0.05629497 0.10419521 0.21200711 0.0196018  0.0498408  0.02125995
 0.06479382 0.04336577 0.09831057 0.07314846 0.0825076  0.10419521
 0.09593582 0.12115382 0.09961959 0.10254433 0.02286235 0.10254433
 0.15248369 0.1003096  0.0825076  0.07005655 0.01897323 0.04990768
 0.11146181 0.05551957 0.55711388 0.04382553 0.04470476 0.03932164
 0.11915

Заметим, что оба метода возвращают вектор длины 35189 (размер нашего словаря).

## Классификация

Воспользуемся изученными методами обработки текстов для решения задачи классификации.

In [17]:
vectorizer = CountVectorizer(encoding='utf8', min_df=5)
vectorizer.fit(texts)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=5,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [18]:
X = vectorizer.transform(texts)

In [19]:
from sklearn.cross_validation import cross_val_score, ShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score

/Users/ekayumov/anaconda/envs/py36/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [20]:
cv = ShuffleSplit(X.shape[0], n_iter=1, test_size=0.3)
for train_ids, test_ids in cv:
    lr = LogisticRegression()
    lr.fit(X[train_ids], target[train_ids])
    preds = lr.predict_proba(X[test_ids])[:,1]
    print('ROC-AUC: %.3f, ACC: %.3f' % (roc_auc_score(target[test_ids], preds), 
                                        accuracy_score(target[test_ids], (preds > 0.5).astype(int))))

ROC-AUC: 0.985, ACC: 0.952


То же самое с tf-idf.

In [21]:
vectorizer = TfidfVectorizer(encoding='utf8', min_df=5)
vectorizer.fit(texts)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=5,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [22]:
X = vectorizer.transform(texts)

In [23]:
cv = ShuffleSplit(X.shape[0], n_iter=1, test_size=0.3)
for train_ids, test_ids in cv:
    lr = LogisticRegression()
    lr.fit(X[train_ids], target[train_ids])
    preds = lr.predict_proba(X[test_ids])[:,1]
    print('ROC-AUC: %.3f, ACC: %.3f' % (roc_auc_score(target[test_ids], preds), 
                                        accuracy_score(target[test_ids], (preds > 0.5).astype(int))))

ROC-AUC: 0.995, ACC: 0.969


## Важность признаков

Как уже упоминалось ранее, веса признаков в линейной модели в случае, если признаки отмасштабированы, характеризуют степень их влияния на значение целевой переменной. В задаче классификации текстов, кроме того, признаки являются хорошо интерпретируемыми, поскольку каждый из них соответствует конкретному слову. Изучим влияние конкретных слов на значение целевой переменной:

In [24]:
f_weights = zip(vectorizer.get_feature_names(), lr.coef_[0])
f_weights = sorted(f_weights, key=lambda i: i[1])
for i in range(1,30):
    print('%s, %.2f' % f_weights[-i])
    
print('...')
for i in reversed(range(1,30)):
    print('%s, %.2f' % f_weights[i])

msg, 1.70
pitt, 1.68
she, 1.66
my, 1.65
health, 1.41
medical, 1.41
doctor, 1.36
com, 1.28
disease, 1.27
of, 1.19
cancer, 1.16
banks, 1.15
is, 1.13
geb, 1.12
in, 1.09
that, 1.06
your, 1.06
photography, 1.05
water, 1.04
gordon, 1.02
he, 1.00
pain, 1.00
her, 0.99
treatment, 0.97
medicine, 0.96
not, 0.95
dyer, 0.95
food, 0.95
portal, 0.93
...
where, -0.98
pov, -0.99
need, -1.01
card, -1.03
pc, -1.03
display, -1.03
hacker, -1.06
ac, -1.09
comp, -1.09
screen, -1.09
color, -1.16
windows, -1.20
points, -1.21
ftp, -1.23
video, -1.24
polygon, -1.27
format, -1.27
software, -1.29
computer, -1.32
bit, -1.35
gif, -1.36
tiff, -1.39
program, -1.44
code, -1.47
file, -1.66
images, -1.67
files, -1.76
3d, -1.93
image, -2.17


## Лемматизация и стемминг

Заметим, что одно и то же слово может встречаться в различных формах (например, "сотрудник" и "сотрудника"), но описанные выше методы интерпретируют их как различные слова, что делает признаковое описание избыточным. Устранить эту проблему можно при помощи **лемматизации** и **стемминга**.

### Стемминг

[**Stemming**](https://en.wikipedia.org/wiki/Stemming) –  это процесс нахождения основы слова. В результате применения данной процедуры однокоренные слова, как правило, преобразуются к одинаковому виду.

**Примеры стемминга:**

| Word        | Stem           |
| ----------- |:-------------:|
| вагон | вагон |
| вагона | вагон |
| вагоне | вагон |
| вагонов | вагон |
| вагоном | вагон |
| вагоны | вагон |
| важная | важн |
| важнее | важн |
| важнейшие | важн |
| важнейшими | важн |
| важничал | важнича |
| важно | важн |

[Snowball](http://snowball.tartarus.org/) – фрэймворк для написания алгоритмов стемминга. Алгоритмы стемминга отличаются для разных языков и используют знания о конкретном языке – списки окончаний для разных чистей речи, разных склонений и т.д. Пример алгоритма для русского языка – [Russian stemming](http://snowballstem.org/algorithms/russian/stemmer.html).

In [25]:
import nltk

In [26]:
stemmer = nltk.stem.snowball.RussianStemmer()
print(stemmer.stem(u'машинное'), stemmer.stem(u'обучение'))

машин обучен


In [27]:
stemmer = nltk.stem.snowball.EnglishStemmer()

def stem_text(text, stemmer):
    tokens = text.split()
    return ' '.join(map(lambda w: stemmer.stem(w), tokens))

stemmed_texts = []
for t in tqdm(texts[:1000]):
    stemmed_texts.append(stem_text(t, stemmer))

100%|██████████| 1000/1000 [00:22<00:00, 45.29it/s]


In [28]:
print(texts[0])

From: dyer@spdcc.com (Steve Dyer)
Subject: Re: Analgesics with Diuretics
Organization: S.P. Dyer Computer Consulting, Cambridge MA

In article <ofk=lve00WB2AvUktO@andrew.cmu.edu> Lawrence Curcio <lc2b+@andrew.cmu.edu> writes:
>I sometimes see OTC preparations for muscle aches/back aches that
>combine aspirin with a diuretic.

You certainly do not see OTC preparations advertised as such.
The only such ridiculous concoctions are nostrums for premenstrual
syndrome, ostensibly to treat headache and "bloating" simultaneously.
They're worthless.

>The idea seems to be to reduce
>inflammation by getting rid of fluid. Does this actually work? 

That's not the idea, and no, they don't work.

-- 
Steve Dyer
dyer@ursa-major.spdcc.com aka {ima,harvard,rayssd,linus,m2c}!spdcc!dyer



In [29]:
print(stemmed_texts[0])

from: dyer@spdcc.com (steve dyer) subject: re: analges with diuret organization: s.p. dyer comput consulting, cambridg ma in articl <ofk=lve00wb2avukto@andrew.cmu.edu> lawrenc curcio <lc2b+@andrew.cmu.edu> writes: >i sometim see otc prepar for muscl aches/back ach that >combin aspirin with a diuretic. you certain do not see otc prepar advertis as such. the onli such ridicul concoct are nostrum for premenstru syndrome, ostens to treat headach and "bloating" simultaneously. they'r worthless. >the idea seem to be to reduc >inflamm by get rid of fluid. doe this actual work? that not the idea, and no, they don't work. -- steve dyer dyer@ursa-major.spdcc.com aka {ima,harvard,rayssd,linus,m2c}!spdcc!dy


Как видим, стеммер работает не очень быстро и запускать его для всей выборки достаточно накладно.

### Лемматизация

[Лемматизация](https://en.wikipedia.org/wiki/Lemmatisation) — процесс приведения слова к его нормальной форме (**лемме**):
- для существительных — именительный падеж, единственное число;
- для прилагательных — именительный падеж, единственное число, мужской род;
- для глаголов, причастий, деепричастий — глагол в инфинитиве.

Например, для русского языка есть библиотека pymorphy2.

In [30]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [31]:
morph.parse('играющих')[0]

Parse(word='играющих', tag=OpencorporaTag('PRTF,impf,tran,pres,actv plur,gent'), normal_form='играть', score=0.16666666666666666, methods_stack=((<DictionaryAnalyzer>, 'играющих', 303, 34),))

Сравним работу стеммера и лемматизатора на примере:

In [32]:
stemmer = nltk.stem.snowball.RussianStemmer()
print(stemmer.stem('играющих'))

игра


In [33]:
print(morph.parse('играющих')[0].normal_form)

играть


## Разреженные матрицы

Разреженная матрица — это матрица, большинство элементов которой равны нулю. Такие матрицы возникают во многих областях науки, в том числе и в машинном обучении.

Для разреженных матриц можно определить следующие характеристики:
- разреженность (sparsity) — доля нулевых элементов матрицы,
- плотность (density) — доля ненулевых элементов матрицы, или $1 - \text(sparsity)$.

Для разреженных матриц существуют специальные способы их хранения в памяти компьютера, при которых хранятся только ненулевые значения, тем самым сокращается объём занимаемой памяти. Эти способы реализованы в библиотеке [scipy.sparse](http://docs.scipy.org/doc/scipy/reference/sparse.html). Кроме того, разреженные матрицы поддерживаются большинством реализаций методов машинного обучения.

In [34]:
import numpy as np
import scipy.sparse as sp

### COOrdinate format

[Координатный формат](http://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.coo_matrix.html#scipy.sparse.coo_matrix) задаёт матрицу при помощи троек (индекс строки, индекс столбца, значение элемента), описывающих ненулевые элементы матрицы. Как правило, тройки сортируют по индексу строки, а затем индексу столбца для ускорения работы. 

Объём занимаемой памяти — $O(n),$ где $n$ — число ненулевых элементов в матрице.

In [36]:
m = (np.arange(9) + 1).reshape(3,3)
print(m)
sparse_m = sp.coo_matrix(m)

[[1 2 3]
 [4 5 6]
 [7 8 9]]


In [37]:
for i in range(len(sparse_m.data)):
    print('(%d, %d, %d)' % (sparse_m.row[i], sparse_m.col[i], sparse_m.data[i]))

(0, 0, 1)
(0, 1, 2)
(0, 2, 3)
(1, 0, 4)
(1, 1, 5)
(1, 2, 6)
(2, 0, 7)
(2, 1, 8)
(2, 2, 9)


Для матрицы, содержащей нулевые элементы, имеем:

In [38]:
m = np.eye(3)*np.arange(1,4)
print(m)
sparse_m = sp.coo_matrix(m)

[[1. 0. 0.]
 [0. 2. 0.]
 [0. 0. 3.]]


In [39]:
for i in range(len(sparse_m.data)):
    print '(%d, %d, %d)' % (sparse_m.row[i], sparse_m.col[i], sparse_m.data[i])

SyntaxError: invalid syntax (<ipython-input-39-272b5a3c2479>, line 2)

### Compressed Sparse Row matrix

[CSR формат](http://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html#scipy.sparse.csr_matrix) - разреженная по строчкам матрица. 

<img src="images/arrays.png">

Формат задаёт матрицу при помощи трёх массивов:
1. $i$-ый элемент первого массива соответствует $i$-ой строке и содержит индекс некоторого элемента во втором массиве,
2. во втором массиве по порядку для каждой строки записаны индексы столбцов ненулевых элементов,
3. третий массив имеет такую же длину, как и второй, и содержит значения соответствующих ненулевых элементов.

Обозначим описанные массивы $a,b,c$. Для получения элемента матрицы на позиции $(i, j)$ необходимо осуществить следующую последовательность действий:
1. Получить значения $a[i]=k_{left}, a[i+1]=k_{right}$.
2. Тогда индексы столбцов ненулевых элементов $i$-ой строки будут находиться в "подмассиве" $b[k_{left}:k_{right}]$.
3. В цикле перебираем элементы подмассива $b[k_{left}:k_{right}]$, пока не встретим элемент, равный $j$.
4. Если такой элемент обнаружен на позиции $m$ (в терминах массива $b$), то ответом является значение $c[m]$.
5. Иначе ответом является 0.если мы не встретили элемент, равный $j$, то возвращаем $0$.

Объём занимаемой памяти — $O(n)$, где $n$ - число ненулевых элементов.

In [ ]:
m = (np.arange(9) + 1).reshape(3,3)
print(m)
sparse_m = sp.csr_matrix(m)

In [ ]:
print('a', sparse_m.indptr)
print('b', sparse_m.indices)
print('c', sparse_m.data)

Для матрицы, содержащей нулевые элементы:

In [ ]:
m = np.tril(np.arange(1,4))
print(m)
sparse_m = sp.csr_matrix(m)

In [ ]:
print('a', sparse_m.indptr)
print('b', sparse_m.indices)
print('c', sparse_m.data)

### Compressed Sparse Column matrix

[CSC формат](http://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csc_matrix.html#scipy.sparse.csc_matrix) - разреженная по столбцам матрица. 

Формат CSC задаёт матрицу аналогично формату CSR, но при этом элементы первого массива соответствуют столбцам, а не строкам.

Объём занимаемой памяти — $O(n)$, где $n$ - число ненулевых элементов.

In [ ]:
m = (np.arange(9) + 1).reshape(3,3)
print(m)
sparse_m = sp.csc_matrix(m)

In [ ]:
print('a', sparse_m.indptr)
print('b', sparse_m.indices)
print('c', sparse_m.data)

In [ ]:
m = np.tril(np.arange(1,4))
print(m)
sparse_m = sp.csc_matrix(m)

In [ ]:
print('a', sparse_m.indptr)
print('b', sparse_m.indices)
print('c', sparse_m.data)

## Умножение разреженных матриц

Как мы убедились, объём занимаемой памяти практически не отличается для всех вариантов хранения разреженных матриц. В таком случае использование какого из вариантов даёт больше преимуществ? Оказывается, что все три способа кардинально различаются по времени умножения матриц.

Для начала вспомним правило умножения матриц:
$$C = A\cdot B$$
$$C_{ij} = \sum_k A_{ik}B_{kj}$$

Для нахождения элемента $C_{ij}$ необходимо получить $i$-ую строчку матрицы $A$ и $j$-ый столбец матрицы $B$. Исследуем время выполнения этих операций для каждого из форматов:

- **COO.** Стоимость получения строки — $O(n)$. Стоимость получения столбца — $O(n)$. При условии, что тройки отсортированы, время поиска можно сократить, воспользовавшись бинарным поиском.
- **CSR.** Стоимость получения строки — $O(1)$. Стоимость получения столбца — $O(n)$.
- **CSC.** Стоимость получения строки — $O(n)$. Стоимость получения столбца — $O(1)$.

Таким образом, время перемножения матриц будет оптимальным, если матрица $A$ задаётся в формате CSR, а матрица $B$ — в формате CSC.

## Разреженные матрицы в линейных моделях

Рассмотрим задачу линейной регрессии с функционалом качества MSE:

$$Q = ||Xw - y||^2 \rightarrow \min_{w}.$$

Как уже говорилось на предыдущих семинарах, вместо нахождения оптимального значения вектора $w$ используют градиентные методы оптимизации функционала. Запишем формулу его градиента:

$$\frac{\partial Q}{\partial w} = 2X^T(Xw - y).$$

Заметим, что матрица $X$, заданная в формате CSR, может быть представлена как $X^T$ в формате CSC (действительно, используя те же массивы, мы можем придать им "симметричный" смысл).

Рассмотрим, как осуществляется умножение разреженной матрицы $A$ на вектор $z$:

1) **CSR**
$$(Az)_{i} = \sum_{k}A_{ik}z_k.$$

Для матрицы в формале CSR обращение к строчкам матрицы выполняется за $O(1)$, поэтому перемножение выполняется за $O(n)$, где $n$ - кол-во ненулевых элементов матрицы $X$.
    
2) **CSC** 

Для матрицы в формате CSC обращение к строчкам матрицы выполняется за $O(n)$. В этом случае умножение будем производить следующим образом:
    - Аллоцируем результирующий вектор, который предварительно заполним нулями. 
    - Обращаемся к $i$-ому столбцу матрицы $A$ и $i$-ому элементу вектора $z$.
    - Каждый ненулевой элемент в столбце домножаем на $z_i$ и добавляем результат к соответствующему значению результирующего вектора.
    
Итого, для умножения разреженной матрицы на вектор получаем следующую асимптотику:
 - $O(l)$ по памяти;
 - $O(n)$ по времени.

Таким образом, мы описали процедуру умножения разреженной матрицы на вектор, и теперь её можно применить для вычисления градиента в задачах с разреженными матрицами "объект-признак".